In [1]:
# Make sure we've got the latest version of fastai:
!pip install -Uqq fastai

In [2]:
from fastai.vision.all import *
from fastai import *


In [3]:
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path
path.ls()

(#3) [Path('train'),Path('valid'),Path('labels.csv')]

In [4]:
#(path/"train").ls()
threes = (path/"train"/"3").ls().sorted()
sevens = (path/"train"/"7").ls().sorted()


In [5]:
im_3 = threes[0]
image_3 = Image.open(im_3)
image_3

In [6]:
image_3t = tensor(image_3)

df = pd.DataFrame(image_3t)

df.style.set_properties(**{"font-size": "6pt"}).background_gradient("Greys")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,42,118,219,166,118,118,6,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,103,242,254,254,254,254,254,66,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,18,232,254,254,254,254,254,238,70,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,104,244,254,224,254,254,254,141,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,207,254,210,254,254,254,34,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,84,206,254,254,254,254,41,0,0,0,0,0,0,0,0,0


In [7]:
three_tensor = [tensor(Image.open(o)) for o in threes]
seven_tensor = [tensor(Image.open(o)) for o in sevens]

print(len(three_tensor))
print(len(seven_tensor))

6131
6265


In [8]:
#Calculate distance between 3 and 7:-

stacked_threes = torch.stack(three_tensor).float()/255
stacked_seven = torch.stack(seven_tensor).float()/255

stacked_threes.shape, stacked_seven.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [9]:
mean3 = stacked_threes.mean(0)
mean7 = stacked_seven.mean(0)

a_3 = stacked_threes[0]
a_7 = stacked_seven[0]

dist_3_L1 = (a_3 - mean3).abs().mean()
dist_3_L2 = ((a_3 - mean3)**2).mean().sqrt()

print(dist_3_L1)
print(dist_3_L2)


tensor(0.1074)
tensor(0.1912)


In [10]:
def dist(a,b):
  res = (a-b).abs().mean((-1,-2))
  return res
#difference b/w mean(0) and mean((-1,-2)) is that mean(0) :-For example, if you have 100 images of the number '3', and you apply mean(0), you would obtain a single averaged '3' image.
#mean((-1,-2)) calculates the mean along the last two dimensions, which are usually the height and width of an image. -1 is last dimension and -2 is second to last dimension.



In [11]:
validation_3 = dist(stacked_threes , mean3)
print(validation_3)
print(validation_3.shape)

tensor([0.1074, 0.1114, 0.1100,  ..., 0.1316, 0.1220, 0.1267])
torch.Size([6131])


In [13]:
import timm

timm.list_models("resnet18")

['resnet18']

In [12]:
def is_3(x):
  return dist(x,mean3)<dist(x,mean7)

print(is_3(stacked_threes))

tensor([True, True, True,  ..., True, True, True])


In [17]:
acc3 = is_3(stacked_threes).float().mean()
acc7 = is_3(stacked_seven).float().mean()

print(f"accuracy of 3 comparing 3s mean :{acc3}")
print(f"accuracy of 7 comparing 3s mean :{acc7}")

accuracy of 3 comparing 3s mean :0.8912085890769958
accuracy of 7 comparing 3s mean :0.0038308061193674803


In [16]:
def is_7(x):
  return dist(x,mean3)>dist(x,mean7)

acc3_1 = is_7(stacked_threes).float().mean()
acc7_1 = is_7(stacked_seven).float().mean()

print(f"accuracy of 3 comparing 7s mean : {acc3_1}")
print(f"accuracy of 7 comparing 7s mean :{acc7_1}")

accuracy of 3 comparing 7s mean : 0.10879138857126236
accuracy of 7 comparing 7s mean :0.9961692094802856
